In [143]:
import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge, LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import statsmodels.formula.api as smf
import seaborn as sns
import json
import heapq
import random

from collections import OrderedDict

def unique_ordered_list(input_list):
    # Use an OrderedDict to maintain order and eliminate duplicates
    unique_dict = OrderedDict.fromkeys(input_list)

    # Get the keys (unique values) from the OrderedDict
    unique_list = list(unique_dict.keys())

    return unique_list

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)


def sim(trck,times,distance,dogs,lower_mult,upper_mult,lower_lim,upper_lim,clear_fave,BSP,few_dogs,use_straight_preds,use_320_preds,use_395_preds,use_520_preds,use_traralgon_preds):
    tracks=[trck]

    under_3=False
    for track in tracks:
        with open('preds.json', 'r') as json_file:
            predictors = json.load(json_file)
        
        if few_dogs == False:
            data=pd.read_csv('class4/'+track+'.csv')
        
        if few_dogs == True:
            data=pd.read_csv('class3/'+track+'.csv')
            
        data=data.dropna(subset=['StartPrice'])
    #     print(np.sum(data.isna()))
        
        if BSP==True:
            data=data.dropna(subset=['BSP'])
        
        overall=[]
        for dist in data['Distance'].unique():
            print(dist)
            #print('CHECK NO. DOGS IN RACE')
            if dist==distance:
                data2=data[data['Distance']==dist]
        #             print(data2.shape[0])
                if(data2.shape[0]>1000):
                    preds= predictors[track][str(dist)]
                    
                    if(use_straight_preds):
                        preds = ['split_z_avg', 'last_5_avg', 'at_cas_dist_time', 'mean_place', 'pir3', 'at_cas_perc', 'collision', 'i1', 'mean_in_race_pos']
                    
                    if(use_320_preds):
                        preds = ['faster_inside','i2','i1','split_z_avg','last_5_avg']
                    
                    if(use_395_preds):
                        preds = ['at_cas_dist_time_avg', 'faster_inside', 'i2', 'i1', 'split_z_avg', 'last_5_avg', 'mean_place', 'time_z_avg']
                    
                    if(use_520_preds):
                        preds = ['at_cas_dist_time_avg', 'faster_inside', 'i2', 'i1', 'split_z_avg', 'last_5_avg', 'mean_place', 'at_cas_perc', 'in_form', 'mean_in_race_pos', 'last_5_avg']
                        
                    if(use_traralgon_preds):
                        preds = ['ctime_z_avg','last_5_avg']
                        
                    
                    print(preds)
                    
                    data2=data2.dropna(subset = preds)

                    races= data2['RaceId'].unique()

                    if(data.shape[0]<50000):
                        threeq = int( 6*data2.shape[0]/12)
                        threeeq = int(data2.shape[0]/3)
                    else:
                        print('here')
                        threeq = int( 10*data2.shape[0]/12)
                        threeeq = int( 2*data2.shape[0]/12)

                    if few_dogs == False:
                        invalid = pd.read_csv('Invalid/Invalid_'+track+'.csv')
                        
                    if few_dogs == True:
                        invalid = pd.read_csv('Invalid3/Invalid'+track+'.csv')
                        
                    invalid = list(invalid['Invalid'])
    #                 print(preds)
    
                    mod = LogisticRegression(random_state=42,max_iter=2000).fit(data2[preds], data2['Win'])

                    a=0
                    for x in range(1,7):
                        if data.shape[0]>50000:
                            data3=data2[a*threeeq:(a+1)*threeeq]
                        else:
                            data3=data2[int(threeeq*a/2):int(threeeq*(a+1)/2)]
                        a+=1
                        
                        if times == True:
                            print(str(list(data3['Date'].head(1)))+'--'+str(list(data3['Date'].tail(1)))) 

                        for strat in ['win']:
                            for upper_price_lim in [100]:
                                for diff_lim in [2]:
                                    wins=0
                                    losses=0
                                    money=0
                                    randmoney=0
                                    betsize=1

                                    ls = list(data3['RaceId'])
                                    unique_ordered_lista = unique_ordered_list(ls)

                                    for raceid in unique_ordered_lista:
                                        if data3[data3['RaceId']==raceid].shape[0] in dogs and raceid not in invalid:                                      
                                            race = data3[data3['RaceId']==raceid]
                                            race=race.sort_values(by='RunTime')
                                            race=race.reset_index()

                                            predicted_times = mod.predict_proba(race[preds])
                                            predicted_times = np.transpose(predicted_times)[1]
                                            predicted_times = predicted_times/sum(predicted_times)
                                            our_prices=1/(predicted_times)
                                            
                                            if BSP==True:
                                                prices = list(race['BSP'])
                                            if BSP==False:
                                                prices = list(race['StartPrice'])

                                            #print(race)
                                            #print(race['Date'][1])
#                                             if race['Date'][1] == '2024-03-15':
#                                                 print(race)
#                                                 print(our_prices)
#                                                 print(prices)

                                            to_bet_on = np.argmin(our_prices)
                                            our_price = our_prices[to_bet_on]
                                            their_price = prices[to_bet_on]
                                            their_favourite = np.argmin(prices)
                                            their_favourite_price = prices[their_favourite]
                                            our_take = our_prices[their_favourite]
                                            our_prices1 = sorted(our_prices)
                                            our_take1 = our_prices1.index(our_take)
                                
                                            our_prices1 = list(our_prices.copy())
                                            our_prices1.remove(our_price)
                                            to_bet_on1 = np.argmin(our_prices1)
                                            our_price1 = our_prices1[to_bet_on1] 
                                            
                                            
                                
                                            if our_price1-our_price>1.3:
                                                clear_fav = True
                                            else:
                                                clear_fav = False
                                            
#                                             if raceid==1002041589:
#                                                 data3[data3['RaceId']==1002041589].to_csv('eeee.csv')

                                            
                                            if (their_price/our_price>lower_mult and their_price/our_price<upper_mult and their_price>lower_lim and their_price<upper_lim and clear_fav == clear_fave):
                                            #if their_favourite != to_bet_on and their_price<3:
#                                                 rand = random.randint(0, 4)
#                                                 their_price_rand = prices[rand]

#                                                 while their_price_rand > 4.5:
#                                                     rand = random.randint(0, 7)
#                                                     their_price_rand = prices[rand]

#                                                 if(rand==0):
#                                                     randmoney+=(their_price_rand-1)*betsize
#                                                 else:
#                                                     randmoney-=betsize

                                                if(to_bet_on==0):
                                                    if BSP == True:
                                                        money+=(their_price-1)*betsize*0.75
                                                        wins+=1
                                                    else:
                                                        money+=(their_price-1)*betsize
                                                        wins+=1



                                                    #print(money)
                                                    #print(their_price)
                                                    #print('---')
                                                    #print(our_price)
                                                    #print(race['Date'][1])
                                                    #print(our_prices)
                                                    #print(our_price)
                                                    #print(prices)
                                                    #print('win')
                                                else:
                                                    money-=betsize
                                                    losses+=1



                                                    #print(money)
                                                    #print(our_price)
                                                    print(race['Date'][1])
                                                    #print(raceid)
                                                    #print(their_price)
                                                    #print(our_prices)
                                                    #print(prices)
                                                    #print('loss')


                                    bets = wins+losses

                                    if(bets!=0):
                                        invested = bets*betsize
                                        roi=100*(money/invested)
                                        win_rate = wins/(wins+losses)


                                    else:
                                        invested=0
                                        roi=0
                                        win_rate = 0

                                    res=[round(roi),round(win_rate,2),bets]
                                    #res=[round(roi),round(win_rate,2),bets,'            ',randmoney]
                                    print(res)
                                    overall.append(res)


            #df = pd.DataFrame(overall)
            #df.columns=['track','dist','strat','max-size','multiplier','roi','winrate','bets']

        
        

In [144]:
#sim(trck,times,distance,dogs,lower_mult,upper_mult,lower_lim,upper_lim,clear_fav,use_bsp,use_straight_preds,use_320_preeds,use_395_preds,use_520_preds,use_traralgon)

sim('rockhampton',False,407,[7,8],   0.1,8,  1.5,2.25,   True,False,False,   False,True,False,False,False)


407
['faster_inside', 'i2', 'i1', 'split_z_avg', 'last_5_avg']
2018-12-13
2019-01-23
2019-02-13
2019-02-27
2019-04-24
2019-06-26
2019-07-16
2019-09-11
[-17, 0.47, 15]
2020-01-21
2020-01-29
2020-04-15
2020-05-13
2020-05-20
2020-06-24
2020-07-15
2020-07-29
2020-09-02
2020-09-23
2020-11-11
2020-12-16
2020-12-23
2021-01-06
2021-01-27
[-37, 0.35, 23]
2021-02-24
2021-04-07
2021-05-19
2021-08-18
2021-08-25
2021-09-01
2021-10-27
2021-11-17
2021-12-01
[18, 0.65, 26]
2022-02-02
2022-03-02
2022-03-16
2022-04-29
2022-05-18
2022-06-01
2022-06-22
2022-06-28
2022-08-26
[16, 0.64, 25]
2022-10-12
2022-10-12
2022-11-23
2023-01-06
2023-04-21
[22, 0.69, 16]
2023-06-07
2023-06-21
2023-07-26
2023-08-09
2023-10-25
2024-01-03
2024-01-31
2024-02-21
2024-03-06
[9, 0.59, 22]
510


In [ ]:
#     import pandas as pd
#     import statistics as st

#     df = pd.read_csv('class/townsville.csv')
#     df['StartPrice']=df['StartPrice'].astype(float,errors='ignore')
#     df['BSP']=df['BSP'].astype(float,errors='ignore')

#     df=df.dropna(subset=['StartPrice','BSP'])

#     x = list(round(df['StartPrice'],2))
#     y = list(round(df['BSP'],2))

#     diffs = []
#     diffs_abs = []
#     for i in range(len(x)):
#         try:
#             if x[i]<4:
#                 diffs.append((y[i]-x[i])/x[i])
#                 diffs_abs.append(abs((y[i]-x[i])/x[i]))
#         except:
#             pass

#     plt.hist(diffs,range(-1,3))

# diffs = []
# for x in range(len(a)):
#     diffs.append(c[x]-b[x])


# mults = []
# mults_avg_sub_3 = []
# mults_avg_over_3 = []
# for i in range(len(c)):
#     mults.append(c[i]/d[i])
#     if d[i]<3:
#         mults_avg_sub_3.append(c[i]/d[i]-1)
#     else:
#         mults_avg_over_3.append(c[i]/d[i]-1)

# print('mean sub 3: '+str(np.mean(mults_avg_sub_3)))
# print('sd sub 3: '+str(np.std(mults_avg_sub_3)))
# print('----------')
# print('mean over 3: '+str(np.mean(mults_avg_over_3)))
# print('sd over 3: '+str(np.std(mults_avg_over_3)))